# 面向目标架构案例实践(中)

核心目标：实现从需求到代码全流程实现

In [184]:
import os
import openai
from openai import OpenAI
import shutil

import json
import io
import inspect
import requests
import re
import random
import string

import openai
from openai import OpenAI
import os

## 初始化客户端
api_key = os.getenv("ZHIPU_API_KEY")
## pip install zhipuai

from zhipuai import ZhipuAI
client = ZhipuAI(api_key=api_key)

In [105]:
def extract_function_code(s_python, detail=0):
    """
    函数提取函数，同时执行函数内容，可以选择打印函数信息
    """
    def extract_code(s_python):
        """
        如果输入的字符串s是一个包含Python代码的Markdown格式字符串，提取出代码部分。
        否则，返回原字符串。

        参数:
        s: 输入的字符串。

        返回:
        提取出的代码部分，或原字符串。
        """
        # 判断字符串是否是Markdown格式
        if '```python' in s_python or 'Python' in s_python or'PYTHON' in s_python:
            # 找到代码块的开始和结束位置
            code_start = s_python.find('def')
            code_end = s_python.find('```\n', code_start)
            # 提取代码部分
            code = s_python[code_start:code_end]
        else:
            # 如果字符串不是Markdown格式，返回原字符串
            code = s_python

        return code
    
    # 提取代码字符串
    code = extract_code(s_python)
    
    # 提取函数名称
    match = re.search(r'def (\w+)', code)
    function_name = match.group(1)
    
    # 将函数写入本地
    with open('%s.py' % function_name, 'w', encoding='utf-8') as f:
        f.write(code)
    
    # 执行该函数
    try:
        exec(code, globals())
    except Exception as e:
        print("An error occurred while executing the code:")
        print(e)
    
    # 打印函数名称
    if detail == 0:
        print("The function name is:%s" % function_name)
    
    if detail == 1:
        with open('%s.py' % function_name, encoding='utf-8') as f:
            content = f.read()
        print(content)

## 1.借助Few-shot完成需求函数的自动编写

```python
get_email_input = "请帮我查下邮箱里最后一封邮件内容。"

get_email_out = "请帮我编写一个python函数，用于查看我的mailbox邮箱中最后一封邮件信息，函数要求如下：\
                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；\
                 3.请将全部功能封装在一个函数内；\
                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"
```

In [5]:
def auto_functions(functions_list):
    """
    Chat模型的functions参数编写函数
    :param functions_list: 包含一个或者多个函数对象的列表；
    :return：满足Chat模型functions参数要求的functions对象
    """
    def functions_generate(functions_list):
        # 创建空列表，用于保存每个函数的描述字典
        functions = []
        # 对每个外部函数进行循环
        for function in functions_list:
            # 读取函数对象的函数说明
            function_description = inspect.getdoc(function)
            # 读取函数的函数名字符串
            function_name = function.__name__
            ## one-shot
            system_prompt1="""输出结果必须是一个JSON格式的字典,举个例子：
            [{'type': 'function', 'function': {'name': 'sunwukong_function', 'description': '该函数定义了数据集计算过程，接收一个数据表字符串参数，并返回一个JSON格式的Dataframe类型对象。', 'parameters': {'type': 'object', 'properties': {'data': {'type': 'string', 'description': '带入计算的数据表，用字符串进行表示'}}, 'required': ['data'], 'additionalProperties': False}}}]
            """
            system_prompt2 = '以下是某的函数说明：%s,输出结果必须是一个JSON格式的字典，只输出这个字典即可，前后不需要任何前后修饰或说明的语句' % function_description
            user_prompt = '根据这个函数的函数说明，请帮我创建一个JSON格式的字典，这个字典有如下5点要求：\
                           1.字典总共有三个键值对；\
                           2.第一个键值对的Key是字符串name，value是该函数的名字：%s，也是字符串；\
                           3.第二个键值对的Key是字符串description，value是该函数的函数的功能说明，也是字符串；\
                           4.第三个键值对的Key是字符串parameters，value是一个JSON Schema对象，用于说明该函数的参数输入规范。\
                           5.输出结果必须是一个JSON格式的字典，只输出这个字典即可，前后不需要任何前后修饰或说明的语句' % function_name

            response = client.chat.completions.create(
                              model="glm-4",
                              messages=[
                                {"role": "system", "content": system_prompt1},
                                {"role": "system", "content": system_prompt2},
                                {"role": "user", "content": user_prompt}
                              ]
                            )
            json_str=response.choices[0].message.content.replace("```json","").replace("```","")
            json_function_description=json.loads(json_str)
            json_str={"type": "function","function":json_function_description}
            functions.append(json_str)
        return functions
    ## 最大可以尝试4次
    max_attempts = 4
    attempts = 0

    while attempts < max_attempts:
        try:
            functions = functions_generate(functions_list)
            break  # 如果代码成功执行，跳出循环
        except Exception as e:
            attempts += 1  # 增加尝试次数
            print("发生错误：", e)
            if attempts == max_attempts:
                print("已达到最大尝试次数，程序终止。")
                raise  # 重新引发最后一个异常
            else:
                print("正在重新运行...")
    return functions

In [2]:
import pymysql
import numpy as np
import pandas as pd

def retrieve_emails(n,user_id='me'):
    """
    获取指定数量的最近邮件。
    参数:
    n: 要检索的邮件的数量,数据格式为int类型。
    user_id: 要检索邮件的用户的ID。默认值是'me'，数据格式为字符串类型。

    返回:
    一个列表，其中每个元素都是一个字典，表示一封邮件。每个字典包含以下键：
    'from_userid': 发件人ID。
    'date': 邮件的发送日期。
    'subject': 邮件的主题。
    'snippet': 邮件的摘要（前100个字符）。
    """
    mysql_pw="gpt"
    
    connection = pymysql.connect(
    host='localhost',  # 数据库地址
    user='gpt',  # 数据库用户名
    passwd=mysql_pw,  # 数据库密码
    db='my_mail',  # 数据库名
    charset='utf8'  # 字符集选择utf8   
    )
    sql_query=f"SELECT * FROM mailbox WHERE userid = '{user_id}' ORDER BY date DESC LIMIT {n}";

    try:
        with connection.cursor() as cursor:
            sql = sql_query
            cursor.execute(sql)
            results = cursor.fetchall()
    finally:
        cursor.close()
    
    column_names = [desc[0] for desc in cursor.description]

    # 使用results和column_names创建DataFrame
    df = pd.DataFrame(results, columns=column_names)
        
    return df.to_json(orient = "records")     

In [3]:
def get_latest_email(userId):
    """
    获取最新的一封邮件
    参数:
    user_id: 要检索邮件的用户的ID。默认值是'me'，数据格式为字符串类型。

    返回:
    一个列表，其中每个元素都是一个字典，表示一封邮件。每个字典包含以下键：
    'from_userid': 发件人ID。
    'date': 邮件的发送日期。
    'subject': 邮件的主题。
    'snippet': 邮件的摘要（前100个字符）。
    """
    mysql_pw="gpt"
    
    connection = pymysql.connect(
    host='localhost',  # 数据库地址
    user='gpt',  # 数据库用户名
    passwd=mysql_pw,  # 数据库密码
    db='my_mail',  # 数据库名
    charset='utf8'  # 字符集选择utf8   
    )
    sql_query=f"SELECT * FROM mailbox WHERE userid = '{user_id}' ORDER BY date DESC LIMIT 1";

    try:
        with connection.cursor() as cursor:
            sql = sql_query
            cursor.execute(sql)
            results = cursor.fetchall()
    finally:
        cursor.close()
    
    column_names = [desc[0] for desc in cursor.description]

    # 使用results和column_names创建DataFrame
    df = pd.DataFrame(results, columns=column_names)
        
    return df.to_json(orient = "records")  

In [4]:
def get_email_count(userId='me'):
    """
    获取指定用户邮箱中的邮件总数。

    参数:
    userId: 要查询的用户的ID。默认值是'me'，数据格式为字符串类型。

    返回:
    一个JSON格式对象，包含一个键值对，键为'count'，值为邮件总数。

    示例:
    {
        "count": 123
    }
    """

    # 数据库连接配置
    config = {
        'host': 'localhost',
        'user': 'gpt',
        'passwd': 'gpt',
        'db': 'my_mail',
        'charset': 'utf8'
    }

    # 建立数据库连接
    try:
        connection = pymysql.connect(**config)
        with connection.cursor() as cursor:
            # 查询邮件总数
            sql = f"SELECT COUNT(*) FROM mailbox WHERE userid = '{userId}'"
            cursor.execute(sql)
            result = cursor.fetchone()
            email_count = result[0] if result else 0

            # 将结果转换为JSON格式
            return json.dumps({'count': email_count})
    except pymysql.MySQLError as e:
        print(f"数据库操作出错: {e}")
        return json.dumps({'error': str(e)})
    finally:
        if connection:
            connection.close()

# 示例调用
# print(get_email_count('me'))

In [35]:
functions_list = [get_latest_email, retrieve_emails,get_email_count]

In [36]:
functions = auto_functions(functions_list)

In [25]:
messages = [{"role": "user", "content": '请帮我看一下邮箱里是否有来自陆小凤的未读邮件，如果有的话请帮我解读下邮件内容。'}]

In [37]:
response = client.chat.completions.create(
                        model="glm-4",
                        messages=messages,
                        tools=functions,
                        tool_choice="auto"
                        )


In [38]:
response

Completion(model='glm-4', created=1717670757, choices=[CompletionChoice(index=0, finish_reason='tool_calls', message=CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_8726257747458047499', function=Function(arguments='{"user_id":"me"}', name='get_latest_email'), type='function', index=0)]))], request_id='8726257747458047499', id='8726257747458047499', usage=CompletionUsage(prompt_tokens=367, completion_tokens=16, total_tokens=383))

我们发现虽然它识别出来需要调用get_latest_email函数，但是其实调用这个函数是解决不了问题的，或者说目前的这些API都支持不了解决这个问题。
需要我们去开发新的API去解决问题。

接下来我们尝试一下，如果让大模型帮我们从需求到代码全流程实现。

为了更加自动的完成这个新增功能的实现过程，我们需要一个提示流程，让Chat模型自行编写用于提示大语言模型创建外部函数的提示词，即我们需要编写一个用于创建提示词的提示词：

In [39]:
get_email_input = "请帮我查下邮箱里最后一封邮件内容。"

get_email_out = "请帮我编写一个python函数，用于查看我的mailbox邮箱中最后一封邮件信息，函数要求如下：\
                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；\
                 3.请将全部功能封装在一个函数内；\
                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

email_counts_input = "请帮我查下邮箱里现在总共有多少封邮件。"

email_counts_out = "请帮我编写一个python函数，用于查看我的mailbox邮箱中总共有多少封邮件，函数要求如下：\
                    1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                    2.函数返回结果是当前邮件总数，返回结果本身必须是一个json格式对象；\
                    3.请将全部功能封装在一个函数内；\
                    4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

In [367]:
# 打开并读取Markdown文件
with open('/root/autodl-tmp/面向目标架构落地/data/my_mail数据字典.md', 'r', encoding='utf-8') as f:
    md_content = f.read()
    
md_content

'# my_mail数据字典：my_mail数据库数据字典\n\n本数据字典记录了my_mail数据库中各张数据表的基本情况。\n我的my_mail中\nhost=\'localhost\',\nuser=\'gpt\',\npasswd="gpt",\ndb=\'my_mail\'\n\n## 1.mailbox数据表\n\n- 基本解释\n\n  \u200b\t\tmailbox数据表记录了邮箱里的信息，包含收件人，发件人，发件时间，邮件主题，邮件内容，邮件状态，邮件标签等。\n\n- 数据来源\n\n  \u200b\tmailbox数据集由系统记录，并且校验过相关信息，数据集的准确性和可信度都非常高。\n\n- 各字段说明\n\n| Column Name | Description | Value Range | Value Explanation | Type |\n|-------------|-------------|-------------|-------------------|------|\n| userid | 收件人 |              | 由字符串组成的 | VARCHAR(255) |\n| from_userid | 发件人 |  | 由字符串组成 | VARCHAR(255) |\n| date | 发邮件的时间 |  | 日期格式  | VARCHAR(255) |\n| subject | 邮件主题 |  | 由字符串组成 | VARCHAR(255) |\n| snippet | 邮件内容 | | 由字符串组成 | VARCHAR(255) |\n| status | 邮件状态 | 已读，未读 | 已读，未读 | VARCHAR(255) |\n| label | 邮件标签 |  | 由字符串组成| VARCHAR(255) |\n\n- 举个例子：\n| userid | from_userid | date | subject | snippet |status|label|\n|-------------|-------------|-------------|-------------------|------|--|--|\n| me | 陆小凤| 2024-07-07|2024年Q2规划|您好，我

In [84]:
user_content = "请查下我的邮箱里是否有来陆小凤的未读邮件，并解读最近一封未读邮件的内容"
# {"role": "system", "content": md_content},
# one-shot  few-shot

messages_fewShot_stage1 = [{"role": "system", "content": f"最后输出的是一个说明，举个例子，输出格式如下：{email_counts_out}"},
                           {"role": "system", "content": "请模仿下列对话给出的风格，基于我的问题，把我的问题转换成一个说明。注意：不需要有额外输出"},
                          {"role": "user", "name":"example1_user", "content": get_email_input},
                          {"role": "assistant", "name":"example1_assistant", "content": get_email_out},
                          {"role": "user", "name":"example2_user", "content": email_counts_input},
                          {"role": "assistant", "name":"example2_assistant", "content": email_counts_out},
                          {"role": "user", "name":"example_user", "content": user_content}]
messages_fewShot_stage1

[{'role': 'system',
  'content': "最后输出的是一个说明，举个例子，输出格式如下：请帮我编写一个python函数，用于查看我的mailbox邮箱中总共有多少封邮件，函数要求如下：                    1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                    2.函数返回结果是当前邮件总数，返回结果本身必须是一个json格式对象；                    3.请将全部功能封装在一个函数内；                    4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
 {'role': 'system', 'content': '请模仿下列对话给出的风格，基于我的问题，把我的问题转换成一个说明。注意：不需要有额外输出'},
 {'role': 'user', 'name': 'example1_user', 'content': '请帮我查下邮箱里最后一封邮件内容。'},
 {'role': 'assistant',
  'name': 'example1_assistant',
  'content': "请帮我编写一个python函数，用于查看我的mailbox邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
 {'role': 'user', 'name': 'example2_user', 'content': '请帮我查下邮箱里现在总共有多少封邮件。'},
 {'role': 'assistant',
  'name': 'example2_assistant',


In [116]:
response = client.chat.completions.create(
                        model="glm-4",
                        messages=messages_fewShot_stage1)

stage1_content = response.choices[0].message.content

In [117]:
print(stage1_content)

请编写以下Python函数，以满足查询邮箱中特定发件人的未读邮件数量，并返回最近一封未读邮件的内容。函数要求如下：

1. 函数名为`check_mailbox_for_specific_sender`。
2. 函数参数`userId`，默认值为字符串`'me'`，表示检查当前用户的邮箱。
3. 函数参数`sender`，字符串类型，用于指定要检查的特定发件人邮箱地址。
4. 函数返回一个JSON格式的对象，包含以下信息：
   - `unread_count`：特定发件人的未读邮件数量。
   - `latest_unread_content`（如果存在未读邮件）：最近一封未读邮件的内容。
5. 请在函数内部添加中文编写的详细函数说明文档，说明函数的功能、参数和返回结果。

以下是对函数说明文档风格的示例：

```python
def check_mailbox_for_specific_sender(userId='me', sender=None):
    """
    检查指定发件人的未读邮件数量，并返回最近一封未读邮件的内容。

    :param userId: 字符串，表示邮箱账户ID，默认为'me'，即当前用户。
    :param sender: 字符串，指定要检查的特定发件人的邮箱地址。
    :return: JSON对象，包含未读邮件数量和最近一封未读邮件的内容（如果存在）。
        {
            "unread_count": int,  # 特定发件人的未读邮件数量
            "latest_unread_content": str or None  # 最近一封未读邮件的内容，如果没有未读邮件则为None
        }

    使用本函数可以方便地跟踪特定联系人的邮件，并快速了解最近一封邮件的内容。
    """
    # 函数实现部分
    # ...
    pass
```

请注意，由于实际邮件服务器的访问权限和安全限制，上述函数的实现需要具体的邮件服务API支持，并且需要处理认证和权限问题。此示例仅提供了函数定义和说明文档的结构。


In [98]:
system_content = md_content

In [99]:
user_example_content = "请帮我编写一个python函数，用于查看我的mailbox邮箱中最后一封邮件信息，函数要求如下：\
                        1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                        2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；\
                        3.请将全部功能封装在一个函数内；\
                        4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

In [95]:
function_name="get_latest_email"
with open(f'./functions/tested functions/{function_name}_module.py',encoding='utf-8') as f:
    assistant_example_content = f.read()

In [96]:
assistant_example_content

'def get_latest_email(userId):\n    """\n    获取最新的一封邮件\n    参数:\n    user_id: 要检索邮件的用户的ID。默认值是\'me\'，数据格式为字符串类型。\n\n    返回:\n    一个列表，其中每个元素都是一个字典，表示一封邮件。每个字典包含以下键：\n    \'from_userid\': 发件人ID。\n    \'date\': 邮件的发送日期。\n    \'subject\': 邮件的主题。\n    \'snippet\': 邮件的摘要（前100个字符）。\n    """\n    mysql_pw="gpt"\n    \n    connection = pymysql.connect(\n    host=\'localhost\',  # 数据库地址\n    user=\'gpt\',  # 数据库用户名\n    passwd=mysql_pw,  # 数据库密码\n    db=\'my_mail\',  # 数据库名\n    charset=\'utf8\'  # 字符集选择utf8   \n    )\n    sql_query=f"SELECT * FROM mailbox WHERE userid = \'{user_id}\' ORDER BY date DESC LIMIT 1";\n\n    try:\n        with connection.cursor() as cursor:\n            sql = sql_query\n            cursor.execute(sql)\n            results = cursor.fetchall()\n    finally:\n        cursor.close()\n    \n    column_names = [desc[0] for desc in cursor.description]\n\n    # 使用results和column_names创建DataFrame\n    df = pd.DataFrame(results, columns=column_names)\n        \n    return df.t

In [118]:
user_content=stage1_content
system_content2="你是一个擅长写代码的智能助手，代码写完后，不需要给代码示例调用。不要有额外的输出"
## 节约Token，各位最好给2个以上的few-shot
messages_fewShot_stage2 = [{"role": "system", "content": system_content},
                           {"role": "system", "content": system_content2},
                          {"role": "user", "name":"example_user", "content": user_example_content},
                          {"role": "assistant", "name":"example_assistant", "content": assistant_example_content},
                          {"role": "user", "name":"example_user", "content": user_content}]

In [119]:
response = client.chat.completions.create(
                        model="glm-4",
                        messages=messages_fewShot_stage2)

stage2_content = response.choices[0].message.content

In [122]:
print(stage2_content)

以下是按照您的要求编写的函数`check_mailbox_for_specific_sender`，它假设您已经有一个数据库连接的设置，并且可以使用SQL查询来检索邮件数据。

```python
import json
import pymysql
from pymysql.cursors import DictCursor

def check_mailbox_for_specific_sender(userId='me', sender=None):
    """
    检查指定发件人的未读邮件数量，并返回最近一封未读邮件的内容。

    :param userId: 字符串，表示邮箱账户ID，默认为'me'，即当前用户。
    :param sender: 字符串，指定要检查的特定发件人的邮箱地址。
    :return: JSON对象，包含未读邮件数量和最近一封未读邮件的内容（如果存在）。
        {
            "unread_count": int,  # 特定发件人的未读邮件数量
            "latest_unread_content": str or None  # 最近一封未读邮件的内容，如果没有未读邮件则为None
        }

    本函数通过连接到my_mail数据库，查询mailbox表来获取特定发件人的邮件信息。
    """

    # 请确保以下变量已经定义并正确设置
    host = 'localhost'
    user = 'gpt'
    password = 'gpt'
    database = 'my_mail'

    # 初始化返回值
    result = {
        "unread_count": 0,
        "latest_unread_content": None
    }

    # 检查输入参数
    if sender is None:
        return json.dumps(result)

    # 建立数据库连接
    connection = pymysql.connect(host, user, password, database, cursorclass=DictCursor)

提取本地代码

In [123]:
extract_function_code(stage2_content, detail=1)

def check_mailbox_for_specific_sender(userId='me', sender=None):
    """
    检查指定发件人的未读邮件数量，并返回最近一封未读邮件的内容。

    :param userId: 字符串，表示邮箱账户ID，默认为'me'，即当前用户。
    :param sender: 字符串，指定要检查的特定发件人的邮箱地址。
    :return: JSON对象，包含未读邮件数量和最近一封未读邮件的内容（如果存在）。
        {
            "unread_count": int,  # 特定发件人的未读邮件数量
            "latest_unread_content": str or None  # 最近一封未读邮件的内容，如果没有未读邮件则为None
        }

    本函数通过连接到my_mail数据库，查询mailbox表来获取特定发件人的邮件信息。
    """

    # 请确保以下变量已经定义并正确设置
    host = 'localhost'
    user = 'gpt'
    password = 'gpt'
    database = 'my_mail'

    # 初始化返回值
    result = {
        "unread_count": 0,
        "latest_unread_content": None
    }

    # 检查输入参数
    if sender is None:
        return json.dumps(result)

    # 建立数据库连接
    connection = pymysql.connect(host, user, password, database, cursorclass=DictCursor)
    try:
        with connection.cursor() as cursor:
            # 查询特定发件人的未读邮件数量
            sql_query_count = f"""
            SELECT COUNT(*) AS unread_count
    

我们使用的这个模型能力比较强，且我们的场景只需要一条SQL语句就解决了，所以大模型生成了正确的代码。
但是如果把给需求给到能力弱一点的模型，或者是不是一个SQL就能解决的场景，代码很有可能就会写错。
我们的需求是：请帮我看一下邮箱里是否有来自陆小凤的未读邮件，如果有的话请帮我解读下邮件内容。
这个需要类比的场景的是，我们干一个事，需要多个步骤，多个参数才能干成。
我们这个函数的核心就是让大模型知道需要传进来2个参数才行，一个参数用来决定查谁的邮箱，一个参数用来查询和谁通信。
所以面对类似稍微复杂一点的需求，我们要想稳定的结果，可以再使用LtM提示流程完成更稳定的函数编写

## 2.借助LtM提示流程完成更稳定的函数编写

```python
get_email_input = "请帮我查下邮箱里最后一封邮件内容。"

get_email_out = "请帮我编写一个python函数，用于查看我的mailbox邮箱中最后一封邮件信息，函数要求如下：\
                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；\
                 3.请将全部功能封装在一个函数内；\
                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"
```

In [ ]:
user_content = "请查下我的邮箱里是否有来陆小凤的未读邮件，并解读最近一封未读邮件的内容"

&emsp;&emsp;而如果在Few-shot的提示下，第一阶段的指令翻译不是每一次都能给出令人满意的结果，首先我们想到的解决问题的办法就是提供一些更加强有力的提示方法：例如LtM提示法。根据LtM多段提示的策略，外加结合当前指令翻译任务考虑，一种更有希望能够引导模型能够顺利完成指令翻译流程的提示方法是：先引导模型拆解当前需求中的“变量”作为后续函数的参数，然后再基于已经确定的变量，引导模型完成翻译工作。其实就整个指令翻译任务来说，最难的地方也就在于函数参数的理解过程，而一旦模型能够准确的翻译出外部函数需要哪些参数，其他部分的翻译任务自然迎刃而解。

In [187]:
system_content1 = "为了更好编写满足用户需求的python函数，我们需要先识别用户需求中的变量，以作为python函数的参数。需要注意的是，当前编写的函数中涉及到的邮件收发查阅等功能，都是通过Python来完成。"

In [188]:
input1 = "请帮我查下mailbox邮箱里最后一封邮件内容。"

In [189]:
pi1 = "当前需求中可以作为函数参数的是：1.查看谁的邮箱。"

In [190]:
input2 = "请帮我给西门吹雪发送一封邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划。"

In [191]:
pi2 = "当前需求中可以作为函数参数的是：1.发送邮件的对象；2.发送邮件的主题；3.邮件具体内容"

In [192]:
input3 = "请查下我的邮箱里是否有来自陆小凤的未读邮件，并解读最近一封未读邮件的内容。"

In [193]:
messages_CD = [{"role": "system", "content": system_content1},
                {"role": "user", "name":"example1_user", "content": input1},
                {"role": "assistant", "name":"example1_assistant", "content": pi1},
                {"role": "user", "name":"example2_user", "content": input2},
                {"role": "assistant", "name":"example2_assistant", "content": pi2},
                {"role": "user", "name":"example_user", "content": input3}]

In [196]:
response = client.chat.completions.create(
                        model="glm-4",
                        messages=messages_CD)

pi3 = response.choices[0].message.content

In [197]:
print(pi3)

当前需求中可以作为函数参数的是：1.需要查看的邮箱；2.发件人；3.未读/已读。


- LtM Command mapping阶段提示

In [198]:
system_content2 = md_content

In [199]:
get_email_input = "请帮我查下邮箱里最后一封邮件内容。" + pi1
get_email_input

'请帮我查下邮箱里最后一封邮件内容。当前需求中可以作为函数参数的是：1.查看谁的邮箱。'

In [200]:
get_email_out = "请帮我编写一个python函数，用于查看我的mailbox邮箱中最后一封邮件信息，函数要求如下：\
                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；\
                 3.请将全部功能封装在一个函数内；\
                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

In [201]:
send_email_input = "请帮我给西门吹雪发送一封mailbox邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划。" + pi2
send_email_input

'请帮我给西门吹雪发送一封mailbox邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划。当前需求中可以作为函数参数的是：1.发送邮件的对象；2.发送邮件的主题；3.邮件具体内容'

In [202]:
send_email_out = "请帮我编写一个python函数，用于给叶孤城发送邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划，函数要求如下：\
                  1.函数参数form_userid、subject和snippet，三个参数都是字符串类型，其中to表示发送邮件对象，subject表示邮件主题，snippet表示邮件具体内容；\
                  2.函数返回结果是当前邮件发送状态，返回结果本身必须是一个json格式对象；\
                  3.请将全部功能封装在一个函数内；\
                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

In [203]:
user_content = input3 + pi3
user_content

'请查下我的邮箱里是否有来自陆小凤的未读邮件，并解读最近一封未读邮件的内容。当前需求中可以作为函数参数的是：1.需要查看的邮箱；2.发件人；3.未读/已读。'

In [213]:
messages_CM = [    {"role": "system", "content": md_content},
                   {"role": "system", "content": "结果里不需要给出示例代码"},
               
                   {"role": "user", "name":"example1_user", "content": get_email_input},
                   {"role": "assistant", "name":"example1_assistant", "content": get_email_out},
                   {"role": "user", "name":"example2_user", "content": send_email_input},
                   {"role": "assistant", "name":"example2_assistant", "content": send_email_out},
                   {"role": "user", "name":"example_user", "content": user_content}]

In [214]:
messages_CM

[{'role': 'system',
  'content': '# my_mail数据字典：my_mail数据库数据字典\n\n本数据字典记录了my_mail数据库中各张数据表的基本情况。\n我的my_mail中\nhost=\'localhost\',\nuser=\'gpt\',\npasswd="gpt",\ndb=\'my_mail\'\n\n## 1.mailbox数据表\n\n- 基本解释\n\n  \u200b\t\tmailbox数据表记录了邮箱里的信息，包含收件人，发件人，发件时间，邮件主题，邮件内容，邮件状态，邮件标签等。\n\n- 数据来源\n\n  \u200b\tmailbox数据集由系统记录，并且校验过相关信息，数据集的准确性和可信度都非常高。\n\n- 各字段说明\n\n| Column Name | Description | Value Range | Value Explanation | Type |\n|-------------|-------------|-------------|-------------------|------|\n| userid | 收件人 |              | 由字符串组成的 | VARCHAR(255) |\n| from_userid | 发件人 |  | 由字符串组成 | VARCHAR(255) |\n| date | 发邮件的时间 |  | 日期格式  | VARCHAR(255) |\n| subject | 邮件主题 |  | 由字符串组成 | VARCHAR(255) |\n| snippet | 邮件内容 | | 由字符串组成 | VARCHAR(255) |\n| status | 邮件状态 | 已读，未读 | 已读，未读 | VARCHAR(255) |\n| label | 邮件标签 |  | 由字符串组成| VARCHAR(255) |'},
 {'role': 'system', 'content': '结果里不需要给出示例代码'},
 {'role': 'user',
  'name': 'example1_user',
  'content': '请帮我查下邮箱里最后一封邮件内容。当前需求中可以作为函数参数的是：1.查看谁的邮箱。'},
 {'r

In [215]:
response = client.chat.completions.create(
                        model="glm-4",
                        messages=messages_CM)

function_description = response.choices[0].message.content

In [216]:
print(function_description)

请帮我编写一个python函数，用于查看我的mailbox邮箱中来自陆小凤的未读邮件，并解读最近一封未读邮件的内容，函数要求如下：                  1.函数参数userId，from_userId和status，其中userId表示需要查看的邮箱，from_userId表示发件人邮箱，status表示邮件状态，未读或已读；                  2.函数返回结果是包含未读邮件内容的对象，如果没有未读邮件，则返回空对象；                  3.请将全部功能封装在一个函数内；                  4.请在函数编写过程中，在函数内部加入详细的函数说明文档，用于说明函数功能、参数情况、返回结果等信息；


- 函数创建

In [217]:
func1_description = get_email_out
func1_description

"请帮我编写一个python函数，用于查看我的mailbox邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

In [218]:
with open('./functions/tested functions/%s_module.py' % 'get_latest_email', encoding='utf-8') as f:
    func1_str = f.read()

In [219]:
func1_str

'def get_latest_email(userId):\n    """\n    获取最新的一封邮件\n    参数:\n    user_id: 要检索邮件的用户的ID。默认值是\'me\'，数据格式为字符串类型。\n\n    返回:\n    一个列表，其中每个元素都是一个字典，表示一封邮件。每个字典包含以下键：\n    \'from_userid\': 发件人ID。\n    \'date\': 邮件的发送日期。\n    \'subject\': 邮件的主题。\n    \'snippet\': 邮件的摘要（前100个字符）。\n    """\n    mysql_pw="gpt"\n    \n    connection = pymysql.connect(\n    host=\'localhost\',  # 数据库地址\n    user=\'gpt\',  # 数据库用户名\n    passwd=mysql_pw,  # 数据库密码\n    db=\'my_mail\',  # 数据库名\n    charset=\'utf8\'  # 字符集选择utf8   \n    )\n    sql_query=f"SELECT * FROM mailbox WHERE userid = \'{user_id}\' ORDER BY date DESC LIMIT 1";\n\n    try:\n        with connection.cursor() as cursor:\n            sql = sql_query\n            cursor.execute(sql)\n            results = cursor.fetchall()\n    finally:\n        cursor.close()\n    \n    column_names = [desc[0] for desc in cursor.description]\n\n    # 使用results和column_names创建DataFrame\n    df = pd.DataFrame(results, columns=column_names)\n        \n    return df.t

In [222]:
system_content3 = "你是一个智能助手，善于基于用户的需求，通过编写函数解决问题，函数参数必须是字符串类型对象，函数返回结果必须是json表示的字符串对象。"

In [220]:
function_description

'请帮我编写一个python函数，用于查看我的mailbox邮箱中来自陆小凤的未读邮件，并解读最近一封未读邮件的内容，函数要求如下：                  1.函数参数userId，from_userId和status，其中userId表示需要查看的邮箱，from_userId表示发件人邮箱，status表示邮件状态，未读或已读；                  2.函数返回结果是包含未读邮件内容的对象，如果没有未读邮件，则返回空对象；                  3.请将全部功能封装在一个函数内；                  4.请在函数编写过程中，在函数内部加入详细的函数说明文档，用于说明函数功能、参数情况、返回结果等信息；'

In [223]:
messages_stage2 = [ {"role": "system", "content": system_content3},
                    {"role": "system", "content": md_content},
                    {"role": "user", "name":"example1_user", "content": func1_description},
                    {"role": "assistant", "name":"example1_assistant", "content": func1_str},
                    {"role": "user","content": function_description}]

In [224]:
response = client.chat.completions.create(
                        model="glm-4",
                        messages=messages_stage2)

python_code = response.choices[0].message.content

In [225]:
print(python_code)

def get_unread_email_content(userId, from_userId, status):
    """
    获取指定邮箱中来自指定发件人且指定状态的未读邮件内容
    参数:
    userId: 邮箱的用户ID，数据格式为字符串类型。
    from_userId: 发件人的用户ID，数据格式为字符串类型。
    status: 邮件的状态，取值为'未读'或'已读'，数据格式为字符串类型。

    返回:
    一个字典对象，表示最近一封满足条件的未读邮件内容。字典包含以下键：
    'from_userid': 发件人ID。
    'date': 邮件的发送日期。
    'subject': 邮件的主题。
    'snippet': 邮件的摘要（前100个字符）。
    如果没有满足条件的未读邮件，则返回空字典。
    """
    mysql_pw = "gpt"
    
    connection = pymysql.connect(
        host='localhost',  # 数据库地址
        user='gpt',  # 数据库用户名
        passwd=mysql_pw,  # 数据库密码
        db='my_mail',  # 数据库名
        charset='utf8'  # 字符集选择utf8   
    )
    
    sql_query = f"SELECT * FROM mailbox WHERE userid = '{userId}' AND from_userid = '{from_userId}' AND status = '{status}' ORDER BY date DESC LIMIT 1"

    try:
        with connection.cursor() as cursor:
            sql = sql_query
            cursor.execute(sql)
            result = cursor.fetchone()
    finally:
        cursor.close()
    
    if result i

In [227]:
extract_function_code(python_code, detail=1)

def get_unread_email_content(userId, from_userId, status):
    """
    获取指定邮箱中来自指定发件人且指定状态的未读邮件内容
    参数:
    userId: 邮箱的用户ID，数据格式为字符串类型。
    from_userId: 发件人的用户ID，数据格式为字符串类型。
    status: 邮件的状态，取值为'未读'或'已读'，数据格式为字符串类型。

    返回:
    一个字典对象，表示最近一封满足条件的未读邮件内容。字典包含以下键：
    'from_userid': 发件人ID。
    'date': 邮件的发送日期。
    'subject': 邮件的主题。
    'snippet': 邮件的摘要（前100个字符）。
    如果没有满足条件的未读邮件，则返回空字典。
    """
    mysql_pw = "gpt"
    
    connection = pymysql.connect(
        host='localhost',  # 数据库地址
        user='gpt',  # 数据库用户名
        passwd=mysql_pw,  # 数据库密码
        db='my_mail',  # 数据库名
        charset='utf8'  # 字符集选择utf8   
    )
    
    sql_query = f"SELECT * FROM mailbox WHERE userid = '{userId}' AND from_userid = '{from_userId}' AND status = '{status}' ORDER BY date DESC LIMIT 1"

    try:
        with connection.cursor() as cursor:
            sql = sql_query
            cursor.execute(sql)
            result = cursor.fetchone()
    finally:
        cursor.close()
    
    if result i

In [228]:
get_unread_email_content

<function __main__.get_unread_email_content(userId, from_userId, status)>

手动测试一下函数

In [ ]:
def get_unread_email_content(userId, from_userId, status):
    """
    获取指定邮箱中来自指定发件人且指定状态的未读邮件内容
    参数:
    userId: 邮箱的用户ID，数据格式为字符串类型。
    from_userId: 发件人的用户ID，数据格式为字符串类型。
    status: 邮件的状态，取值为'未读'或'已读'，数据格式为字符串类型。

    返回:
    一个字典对象，表示最近一封满足条件的未读邮件内容。字典包含以下键：
    'from_userid': 发件人ID。
    'date': 邮件的发送日期。
    'subject': 邮件的主题。
    'snippet': 邮件的摘要（前100个字符）。
    如果没有满足条件的未读邮件，则返回空字典。
    """
    mysql_pw = "gpt"
    
    connection = pymysql.connect(
        host='localhost',  # 数据库地址
        user='gpt',  # 数据库用户名
        passwd=mysql_pw,  # 数据库密码
        db='my_mail',  # 数据库名
        charset='utf8'  # 字符集选择utf8   
    )
    
    sql_query = f"SELECT * FROM mailbox WHERE userid = '{userId}' AND from_userid = '{from_userId}' AND status = '{status}' ORDER BY date DESC LIMIT 1"

    try:
        with connection.cursor() as cursor:
            sql = sql_query
            cursor.execute(sql)
            result = cursor.fetchone()
    finally:
        cursor.close()
    
    if result is not None:
        column_names = [desc[0] for desc in cursor.description]
        email_content = dict(zip(column_names, result))
        return email_content
    else:
        return {}

In [323]:
get_unread_email_content(userId='me', from_userId='陆小凤', status= '未读')

{'userid': 'me',
 'from_userid': '陆小凤',
 'date': '2024-06-03',
 'subject': '2024年Q2规划',
 'snippet': 'Q2规划：聚焦核心业务，优化流程，提升效率。加强团队协作，培养人才。探索新市场，拓宽业务领域。确保目标达成，实现增长。详细见附件',
 'status': '未读',
 'label': '规划'}

- 流程测试

In [339]:
import os
import openai
from openai import OpenAI
import shutil

import json
import io
import inspect
import requests
import re
import random
import string

## 初始化客户端
api_key = os.getenv("ZHIPU_API_KEY")
## pip install zhipuai

from zhipuai import ZhipuAI
client = ZhipuAI(api_key=api_key)

In [340]:
def auto_functions(functions_list):
    """
    Chat模型的functions参数编写函数
    :param functions_list: 包含一个或者多个函数对象的列表；
    :return：满足Chat模型functions参数要求的functions对象
    """
    def functions_generate(functions_list):
        # 创建空列表，用于保存每个函数的描述字典
        functions = []
        # 对每个外部函数进行循环
        for function in functions_list:
            # 读取函数对象的函数说明
            function_description = inspect.getdoc(function)
            # 读取函数的函数名字符串
            function_name = function.__name__
            system_prompt1="""输出结果必须是一个JSON格式的字典,举个例子：
            [{'type': 'function', 'function': {'name': 'sunwukong_function', 'description': '该函数定义了数据集计算过程，接收一个数据表字符串参数，并返回一个JSON格式的Dataframe类型对象。', 'parameters': {'type': 'object', 'properties': {'data': {'type': 'string', 'description': '带入计算的数据表，用字符串进行表示'}}, 'required': ['data'], 'additionalProperties': False}}}]
            """
            system_prompt2 = '以下是某的函数说明：%s,输出结果必须是一个JSON格式的字典，只输出这个字典即可，前后不需要任何前后修饰或说明的语句' % function_description
            user_prompt = '根据这个函数的函数说明，请帮我创建一个JSON格式的字典，这个字典有如下5点要求：\
                           1.字典总共有三个键值对；\
                           2.第一个键值对的Key是字符串name，value是该函数的名字：%s，也是字符串；\
                           3.第二个键值对的Key是字符串description，value是该函数的函数的功能说明，也是字符串；\
                           4.第三个键值对的Key是字符串parameters，value是一个JSON Schema对象，用于说明该函数的参数输入规范。\
                           5.输出结果必须是一个JSON格式的字典，只输出这个字典即可，前后不需要任何前后修饰或说明的语句' % function_name

            response = client.chat.completions.create(
                              model="glm-4",
                              messages=[
                                {"role": "system", "content": system_prompt1},
                                {"role": "system", "content": system_prompt2},
                                {"role": "user", "content": user_prompt}
                              ]
                            )
            json_str=response.choices[0].message.content.replace("```json","").replace("```","")
            json_function_description=json.loads(json_str)
            json_str={"type": "function","function":json_function_description}
            functions.append(json_str)
        return functions
    ## 最大可以尝试4次
    max_attempts = 4
    attempts = 0

    while attempts < max_attempts:
        try:
            functions = functions_generate(functions_list)
            break  # 如果代码成功执行，跳出循环
        except Exception as e:
            attempts += 1  # 增加尝试次数
            print("发生错误：", e)
            if attempts == max_attempts:
                print("已达到最大尝试次数，程序终止。")
                raise  # 重新引发最后一个异常
            else:
                print("正在重新运行...")
    return functions

In [341]:
functions_list = [get_unread_email_content]

In [342]:
functions = auto_functions(functions_list)

发生错误： Expecting value: line 22 column 29 (char 543)
正在重新运行...


In [343]:
functions

[{'type': 'function',
  'function': {'name': 'get_unread_email_content',
   'description': '获取指定邮箱中来自指定发件人且指定状态的未读邮件内容',
   'parameters': {'type': 'object',
    'properties': {'userId': {'type': 'string',
      'description': '邮箱的用户ID，数据格式为字符串类型'},
     'from_userId': {'type': 'string', 'description': '发件人的用户ID，数据格式为字符串类型'},
     'status': {'type': 'string',
      'description': "邮件的状态，取值为'未读'或'已读'，数据格式为字符串类型"}},
    'required': ['userId', 'from_userId', 'status'],
    'additionalProperties': False}}}]

In [344]:
messages = [{"role": "user", "content": '请查下我的邮箱里是否有来自陆小凤的未读邮件，并解读最近一封未读邮件的内容。'}]
response = client.chat.completions.create(
                        model="glm-4",
                        messages=messages,
                        tools=functions,
                        tool_choice="auto"
                        )


In [345]:
response

Completion(model='glm-4', created=1717737110, choices=[CompletionChoice(index=0, finish_reason='tool_calls', message=CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_8726260736755248628', function=Function(arguments='{"from_userId":"陆小凤","status":"未读","userId":"我的"}', name='get_unread_email_content'), type='function', index=0)]))], request_id='8726260736755248628', id='8726260736755248628', usage=CompletionUsage(prompt_tokens=234, completion_tokens=30, total_tokens=264))

In [374]:
def run_conversation(messages, functions_list=None, model="glm-4"):
    """
    能够自动执行外部函数调用的对话模型
    :param messages: 必要参数，字典类型，输入到Chat模型的messages参数对象
    :param functions_list: 可选参数，默认为None，可以设置为包含全部外部函数的列表对象
    :param model: Chat模型，可选参数，默认模型为glm-4
    :return：Chat模型输出结果
    """
    # 如果没有外部函数库，则执行普通的对话任务
    if functions_list == None:
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        )
        response_message = response.choices[0].message
        final_response = response_message.content
        
    # 若存在外部函数库，则需要灵活选取外部函数并进行回答
    else:
        # 创建functions对象
        tools = auto_functions(functions_list)
        
        # 创建外部函数库字典
        available_functions = {func.__name__: func for func in functions_list}
        print(available_functions)

        # 第一次调用大模型
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        tools=tools,
                        tool_choice="auto", )
        response_message = response.choices[0].message
        
       


        tool_calls = response_message.tool_calls

        if tool_calls:

            #messages.append(response.choices[0].message)
            ## GLM-4 Function Call的使用方式
            messages.append(response.choices[0].message.model_dump())
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                ## 真正执行外部函数的就是这儿的代码
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "role": "tool",
                        "content": function_response,
                        "tool_call_id": tool_call.id,
                    }
                ) 
            ## 第二次调用模型
            second_response = client.chat.completions.create(
                model=model,
                messages=messages,
                tools=tools
            ) 
            # 获取最终结果
            print(second_response.choices[0].message)
            final_response = second_response.choices[0].message.content
        else:
            final_response = response_message.content
                
    return final_response

In [375]:
messages = [
     {"role": "system", "content": md_content},
     {"role": "system", "content": "我的邮箱userId叫me,陆小凤的邮箱id就叫陆小凤"},
     {"role": "user", "content": '请查下我的邮箱里来自陆小凤的未读邮件，并解读最近一封未读邮件的内容。'}
]
run_conversation(messages, functions_list=functions_list, model="glm-4")

{'get_unread_email_content': <function get_unread_email_content at 0x7f4df90f1e50>}
CompletionMessage(content='me的邮箱里来自陆小凤的未读邮件已经找到，最近一封未读邮件的内容是：“您好，我是陆小凤，这封邮件是关于2024年Q2规划的讨论，请您在开会前阅读。”', role='assistant', tool_calls=None)


'me的邮箱里来自陆小凤的未读邮件已经找到，最近一封未读邮件的内容是：“您好，我是陆小凤，这封邮件是关于2024年Q2规划的讨论，请您在开会前阅读。”'

- 测试多轮对话的效果

In [398]:
import pymysql
import numpy as np
import pandas as pd
# 这个是自己手写的。
def retrieve_emails(n,user_id='me'):
    """
    获取指定数量的新邮件。
    参数:
    n: 要检索的邮件的数量,数据格式为int类型。
    user_id: 要检索邮件的用户的ID。默认值是'me'，数据格式为字符串类型。

    返回:
    一个列表，其中每个元素都是一个字典，表示一封邮件。每个字典包含以下键：
    'from_userid': 发件人ID。
    'date': 邮件的发送日期。
    'subject': 邮件的主题。
    'snippet': 邮件的摘要（前100个字符）。
    """
    mysql_pw="gpt"
    
    connection = pymysql.connect(
    host='localhost',  # 数据库地址
    user='gpt',  # 数据库用户名
    passwd=mysql_pw,  # 数据库密码
    db='my_mail',  # 数据库名
    charset='utf8'  # 字符集选择utf8   
    )
    sql_query=f"SELECT * FROM mailbox WHERE userid = '{user_id}' ORDER BY date DESC LIMIT {n}";

    try:
        with connection.cursor() as cursor:
            sql = sql_query
            cursor.execute(sql)
            results = cursor.fetchall()
    finally:
        cursor.close()
    
    column_names = [desc[0] for desc in cursor.description]

    # 使用results和column_names创建DataFrame
    df = pd.DataFrame(results, columns=column_names)
        
    return df.to_json(orient = "records")     

In [377]:
## 这个是自动生成的
def get_latest_email(userId):
    """
    获取最新的一封邮件
    参数:
    user_id: 要检索邮件的用户的ID。默认值是'me'，数据格式为字符串类型。

    返回:
    一个列表，其中每个元素都是一个字典，表示一封邮件。每个字典包含以下键：
    'from_userid': 发件人ID。
    'date': 邮件的发送日期。
    'subject': 邮件的主题。
    'snippet': 邮件的摘要（前100个字符）。
    """
    mysql_pw="gpt"
    
    connection = pymysql.connect(
    host='localhost',  # 数据库地址
    user='gpt',  # 数据库用户名
    passwd=mysql_pw,  # 数据库密码
    db='my_mail',  # 数据库名
    charset='utf8'  # 字符集选择utf8   
    )
    sql_query=f"SELECT * FROM mailbox WHERE userid = '{user_id}' ORDER BY date DESC LIMIT 1";

    try:
        with connection.cursor() as cursor:
            sql = sql_query
            cursor.execute(sql)
            results = cursor.fetchall()
    finally:
        cursor.close()
    
    column_names = [desc[0] for desc in cursor.description]

    # 使用results和column_names创建DataFrame
    df = pd.DataFrame(results, columns=column_names)
        
    return df.to_json(orient = "records")  

In [378]:
## 这个是自动生成的
def get_email_count(userId='me'):
    """
    获取指定用户邮箱中的邮件总数。

    参数:
    userId: 要查询的用户的ID。默认值是'me'，数据格式为字符串类型。

    返回:
    一个JSON格式对象，包含一个键值对，键为'count'，值为邮件总数。

    示例:
    {
        "count": 123
    }
    """

    # 数据库连接配置
    config = {
        'host': 'localhost',
        'user': 'gpt',
        'passwd': 'gpt',
        'db': 'my_mail',
        'charset': 'utf8'
    }

    # 建立数据库连接
    try:
        connection = pymysql.connect(**config)
        with connection.cursor() as cursor:
            # 查询邮件总数
            sql = f"SELECT COUNT(*) FROM mailbox WHERE userid = '{userId}'"
            cursor.execute(sql)
            result = cursor.fetchone()
            email_count = result[0] if result else 0

            # 将结果转换为JSON格式
            return json.dumps({'count': email_count})
    except pymysql.MySQLError as e:
        print(f"数据库操作出错: {e}")
        return json.dumps({'error': str(e)})
    finally:
        if connection:
            connection.close()

# 示例调用
# print(get_email_count('me'))

In [394]:
## 这个是自动生成的
def get_unread_email_content(userId, from_userId, status):
    """
    获取指定邮箱中最近的来自指定发件人且指定状态的未读邮件内容
    参数:
    userId: 邮箱的用户ID，数据格式为字符串类型。
    from_userId: 发件人的用户ID，数据格式为字符串类型。
    status: 邮件的状态，取值为'未读'或'已读'，数据格式为字符串类型。

    返回:
    一个字典对象，表示最近一封满足条件的未读邮件内容。字典包含以下键：
    'from_userid': 发件人ID。
    'date': 邮件的发送日期。
    'subject': 邮件的主题。
    'snippet': 邮件的摘要（前100个字符）。
    如果没有满足条件的未读邮件，则返回空字典。
    """
    mysql_pw = "gpt"
    
    connection = pymysql.connect(
        host='localhost',  # 数据库地址
        user='gpt',  # 数据库用户名
        passwd=mysql_pw,  # 数据库密码
        db='my_mail',  # 数据库名
        charset='utf8'  # 字符集选择utf8   
    )
    
    sql_query = f"SELECT * FROM mailbox WHERE userid = '{userId}' AND from_userid = '{from_userId}' AND status = '{status}' ORDER BY date DESC LIMIT 1"

    try:
        with connection.cursor() as cursor:
            sql = sql_query
            cursor.execute(sql)
            result = cursor.fetchone()
    finally:
        cursor.close()
    
    if result is not None:
        column_names = [desc[0] for desc in cursor.description]
        email_content = dict(zip(column_names, result))
        return email_content
    else:
        return {}

In [395]:
functions_list = [retrieve_emails, get_email_count, get_unread_email_content]

In [396]:
def chat_with_model(functions_list=None, 
                    prompt="你好", 
                    model="glm-4", 
                    system_message=[{"role": "system", "content": "你叫小智"}]):
    
    messages = system_message
    messages.append({"role": "user", "content": prompt})
    
    while True:           
        answer = run_conversation(messages=messages, 
                                    functions_list=functions_list, 
                                    model=model)
        
        print(f"智能助手回答: {answer}")
        
        
        # 询问用户是否还有其他问题
        user_input = input("您还有其他问题吗？(输入退出以结束对话): ")
        if user_input == "退出":
            break

        # 记录用户回答
        messages.append({"role": "user", "content": user_input})

这个while循环里run_conversation方法，里面每次都会把所有函数都初始化一次，大家可以把这儿单独优化一下哈。
不需要每次都初始化。

In [387]:
user_prompt="请查下me的邮箱里来自陆小凤的未读邮件，并解读最近一封未读邮件的内容。"

In [399]:

system_message=[
    {"role": "system", "content":md_content},
    {"role": "system", "content": "我的邮箱userId叫me,陆小凤的邮箱id就叫陆小凤，西门吹雪的邮箱id就叫西门吹雪"}
    ]
chat_with_model(functions_list=functions_list, prompt=user_prompt,
                system_message=system_message)

{'retrieve_emails': <function retrieve_emails at 0x7f4df80cfc10>, 'get_email_count': <function get_email_count at 0x7f4df80cfaf0>, 'get_unread_email_content': <function get_unread_email_content at 0x7f4df809a1f0>}
CompletionMessage(content='me的邮箱里来自陆小凤的未读邮件已经找到，最近一封未读邮件的内容是：“您好，我是陆小凤。这封邮件是关于2024年Q2规划的，我希望能够和你讨论一下我们的计划。请尽快回复我，谢谢。”', role='assistant', tool_calls=None)
智能助手回答: me的邮箱里来自陆小凤的未读邮件已经找到，最近一封未读邮件的内容是：“您好，我是陆小凤。这封邮件是关于2024年Q2规划的，我希望能够和你讨论一下我们的计划。请尽快回复我，谢谢。”


您还有其他问题吗？(输入退出以结束对话):  退出


## 3. 全自动功能函数编程

### 3.1 提示词管理

In [512]:
# 第一阶段LtM_CD阶段提示词及输出结果
get_latest_email_CD_input = "请帮我查下mailbox邮箱里最新一封邮件内容。"
get_latest_email_pi = "当前需求中可以作为函数参数的是：1.查看谁的邮箱。"
get_latest_email_messages_CD = [
                                {"role": "user", "content": get_latest_email_CD_input},
                                {"role": "assistant", "content": get_latest_email_pi}
                                ]

# 第一阶段LtM_CM阶段提示词及输出结果
get_latest_email_CM_input = get_latest_email_CD_input + get_latest_email_pi
get_latest_email_description = "请帮我编写一个python函数，用于查看我的mailbox邮箱中最后一封邮件信息，函数要求如下：\
                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；\
                 3.请将全部功能封装在一个函数内；\
                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"
get_latest_email_messages_CM = [
                                {"role": "user", "content": get_latest_email_CM_input},
                                {"role": "assistant", "content":get_latest_email_description}
                                ]

# 第二阶段提示词及输出结果
with open('./functions/tested functions/%s/%s_module.py' % ('get_latest_email', 'get_latest_email'), encoding='utf-8') as f:
    get_latest_email_function = f.read()
get_latest_email_messages = [
                             {"role": "user", "content": get_latest_email_description},
                             {"role": "assistant", "content":get_latest_email_function}
                             ]  

In [513]:
get_latest_email_prompt = {
                            "stage1_CD": get_latest_email_messages_CD,
                            "stage1_CM": get_latest_email_messages_CM,
                            "stage2": get_latest_email_messages
                          }   

In [517]:
get_latest_email_prompt

{'stage1_CD': [{'role': 'user', 'content': '请帮我查下mailbox邮箱里最新一封邮件内容。'},
  {'role': 'assistant', 'content': '当前需求中可以作为函数参数的是：1.查看谁的邮箱。'}],
 'stage1_CM': [{'role': 'user',
   'content': '请帮我查下mailbox邮箱里最新一封邮件内容。当前需求中可以作为函数参数的是：1.查看谁的邮箱。'},
  {'role': 'assistant',
   'content': "请帮我编写一个python函数，用于查看我的mailbox邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"}],
 'stage2': [{'role': 'user',
   'content': "请帮我编写一个python函数，用于查看我的mailbox邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
  {'role': 'assistant',
   'content': 'def get_latest_email(userId):\n    """\n    获取最新的一封邮件\n  

In [518]:
with open('./functions/tested functions/%s/%s_prompt.json' % ('get_latest_email','get_latest_email'), 'w') as f:
    json.dump(get_latest_email_prompt, f)

In [515]:
with open('./functions/tested functions/%s/%s_prompt.json' % ('get_latest_email','get_latest_email'), 'r') as f:
    data = json.load(f)

In [498]:
data

{'stage1_CD': [{'role': 'user', 'content': '请帮我查下mailbox邮箱里最新一封邮件内容。'},
  {'role': 'assistant', 'content': '当前需求中可以作为函数参数的是：1.查看谁的邮箱。'}],
 'stage1_CM': [{'role': 'user',
   'content': '请帮我查下mailbox邮箱里最新一封邮件内容。当前需求中可以作为函数参数的是：1.查看谁的邮箱。'},
  {'role': 'assistant',
   'content': "请帮我编写一个python函数，用于查看我的mailbox邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"}],
 'stage2': [{'role': 'user',
   'content': "请帮我编写一个python函数，用于查看我的mailbox邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
  {'role': 'assistant',
   'content': 'def get_latest_email(userId):\n    """\n    获取最新的一封邮件\n  

In [522]:
retrieve_emails_CD_input = "请帮我查看下最近3封邮件的邮件内容"
retrieve_emails_pi = "当前需求中可以作为函数参数的是：1.查询几封邮件；2.查询谁的邮箱"
retrieve_emails_messages_CD = [
                          {"role": "user", "content": retrieve_emails_CD_input},
                          {"role": "assistant", "content": retrieve_emails_pi}
                         ]

retrieve_emails_CM_input = retrieve_emails_CD_input + retrieve_emails_pi
retrieve_emails_description = "请帮我编写一个python函数，查询我的邮箱里面最近的几封邮件信息，函数要求如下：\
                  1.函数参数n和user_id，两个参数都是字符串类型，其中n表示查询最近的几封邮件，user_id表示要检索邮件的用户的ID。默认值是'me'，表示当前授权的用户；\
                  2.函数返回结果是查询到的这几封邮件的邮件内容，返回结果本身必须是一个json格式对象；\
                  3.请将全部功能封装在一个函数内；\
                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"
retrieve_emails_messages_CM = [
                          {"role": "user", "content": retrieve_emails_CM_input},
                          {"role": "assistant", "content":retrieve_emails_description}
                         ]

with open('./functions/tested functions/%s/%s_module.py' % ('retrieve_emails','retrieve_emails'), encoding='utf-8') as f:
    retrieve_emails_function = f.read()
retrieve_emails_messages = [
                       {"role": "user", "content": retrieve_emails_description},
                       {"role": "assistant", "content":retrieve_emails_function}
                      ] 

In [523]:
retrieve_emails_prompt = {
                     "stage1_CD": retrieve_emails_messages_CD,
                     "stage1_CM": retrieve_emails_messages_CM,
                     "stage2": retrieve_emails_messages
                    }   

In [524]:
with open('./functions/tested functions/%s/%s_prompt.json' % ('retrieve_emails','retrieve_emails'), 'w') as f:
    json.dump(retrieve_emails_prompt, f)

In [525]:
get_email_counts_CD_input = "请帮我查下邮箱里面总共有多少封邮件。"
get_email_counts_pi = "当前需求中可以作为函数参数的是：1.查询谁的邮箱"
get_email_counts_messages_CD = [
                          {"role": "user", "content": get_email_counts_CD_input},
                          {"role": "assistant", "content": get_email_counts_pi}
                         ]

get_email_counts_CM_input = get_email_counts_CD_input + get_email_counts_pi
get_email_counts_description = "请帮我编写一个python函数，用于查看我的mailbox邮箱中总共有多少封邮件，函数要求如下：\
                  1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                  2.函数返回结果是查询到的这几封邮件的邮件内容，返回结果本身必须是一个json格式对象；\
                  3.请将全部功能封装在一个函数内；\
                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"
get_email_counts_messages_CM = [
                          {"role": "user", "content": get_email_counts_CM_input},
                          {"role": "assistant", "content":get_email_counts_description}
                         ]

with open('./functions/tested functions/%s/%s_module.py' % ('get_email_count','get_email_count'), encoding='utf-8') as f:
    get_email_counts_function = f.read()
get_email_counts_messages = [
                       {"role": "user", "content": get_email_counts_description},
                       {"role": "assistant", "content":get_email_counts_function}
                      ] 

In [526]:
get_email_counts_prompt = {
                     "stage1_CD": get_email_counts_messages_CD,
                     "stage1_CM": get_email_counts_messages_CM,
                     "stage2": get_email_counts_messages
                    }   

In [528]:
with open('./functions/tested functions/%s/%s_prompt.json' % ('get_email_count','get_email_count'), 'w') as f:
    json.dump(get_email_counts_prompt, f)

- 系统提示消息保存

我们当前提示流程中各阶段的系统提示消息，也需要对其进行整理和本地保存：

In [529]:
system_messages = {"system_message_CD": [{"role": "system", "content": md_content}], 
                   "system_message_CM": [{"role": "system", "content":md_content }], 
                   "system_message": [{"role": "system", "content":md_content}]}

In [530]:
with open('./functions/tested functions/%s.json' % 'system_messages', 'w') as f:
    json.dump(system_messages, f)

- 定义提示词管理的辅助函数

In [417]:
def remove_to_tested(function_name):
    """
    将函数同名文件夹由untested文件夹转移至tested文件夹内。\
    完成转移则说明函数通过测试，可以使用。
    """
    
    # 将函数代码写入glmLesson.py文件中
    with open('./functions/untested functions/%s/%s_module.py' % (function_name, function_name), encoding='utf-8') as f:
        function_code = f.read()
    
    # 源文件夹路径
    src_dir = './functions/untested functions/%s' % function_name

    # 目标文件夹路径
    dst_dir = './functions/tested functions/%s' % function_name
    
    # 移动文件夹
    shutil.move(src_dir, dst_dir)

In [418]:
def extract_function_code(s, detail=0, tested=False, g=globals()):
    """
    函数提取函数，同时执行函数内容，可以选择打印函数信息，并选择代码保存的地址
    """
    def extract_code(s):
        """
        如果输入的字符串s是一个包含Python代码的Markdown格式字符串，提取出代码部分。
        否则，返回原字符串。

        参数:
        s: 输入的字符串。

        返回:
        提取出的代码部分，或原字符串。
        """
        # 判断字符串是否是Markdown格式
        if '```python' in s or 'Python' in s or'PYTHON' in s:
            # 找到代码块的开始和结束位置
            code_start = s.find('def')
            code_end = s.find('```\n', code_start)
            # 提取代码部分
            code = s[code_start:code_end]
        else:
            # 如果字符串不是Markdown格式，返回原字符串
            code = s

        return code
    
    # 提取代码字符串
    code = extract_code(s)
    
    # 提取函数名称
    match = re.search(r'def (\w+)', code)
    function_name = match.group(1)
    
    # 在untested文件夹内创建函数同名文件夹
    directory = './functions/untested functions/%s' % function_name
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # 将函数写入本地
    if tested == False:
        with open('./functions/untested functions/%s/%s_module.py' % (function_name, function_name), 'w', encoding='utf-8') as f:
            f.write(code)
    else:
        # 调用remove_to_test函数将函数文件夹转移至tested文件夹内
        remove_to_tested(function_name)
        with open('./functions/tested functions/%s/%s_module.py' % (function_name, function_name), 'w', encoding='utf-8') as f:
            f.write(code)
    
    # 执行该函数
    try:
        exec(code, g)
    except Exception as e:
        print("An error occurred while executing the code:")
        print(e)
    
    # 打印函数名称
    if detail == 0:
        print("The function name is:%s" % function_name)
    
    if detail == 1:
        if tested == False:
            with open('./functions/untested functions/%s/%s_module.py' % (function_name, function_name), 'r', encoding='utf-8') as f:
                content = f.read()
        else:
            with open('./functions/tested functions/%s/%s_module.py' % (function_name, function_name), 'r', encoding='utf-8') as f:   
                content = f.read()
                
        print(content)
        
    return function_name

In [423]:
def show_functions(tested=False, if_print=False):
    """
    打印tested或untested文件夹内全部函数
    """
    current_directory = os.getcwd()
    if tested == False:
        directory = current_directory + '/functions/untested functions'
    else:
        directory = current_directory + '/functions/tested functions'
    files_and_directories = os.listdir(directory)
    # 过滤结果，只保留.py文件和非__pycache__文件夹
    files_and_directories = files_and_directories = [name for name in files_and_directories if (os.path.splitext(name)[1] == '.py' or os.path.isdir(os.path.join(directory, name))) and name != ".ipynb_checkpoints"]
    
    if if_print != False:
        for name in files_and_directories:
            print(name)
    
    return files_and_directories

In [531]:
show_functions(tested=True)

['get_latest_email', 'get_email_count', 'retrieve_emails']

并且，我们可以使用如下方法将每个函数名称同名字符串转化为函数：

In [532]:
globals()['get_email_count']

<function __main__.get_email_count(userId='me')>

In [461]:
[globals()[name] for name in show_functions(tested=True)]

[<function __main__.get_latest_email(userId)>,
 <function __main__.get_email_count(userId='me')>,
 <function __main__.retrieve_emails(n, user_id='me')>]

## 3.2 code_generate全自动编程函数创建方法

In [557]:
def code_generate(req, few_shot='all', model='glm-4', g=globals(), detail=0):
    """
    Function calling外部函数自动创建函数，可以根据用户的需求，直接将其翻译为Chat模型可以直接调用的外部函数代码。
    :param req: 必要参数，字符串类型，表示输入的用户需求；
    :param few_shot: 可选参数，默认取值为字符串all，用于描述Few-shot提示示例的选取方案，当输入字符串all时，则代表提取当前外部函数库中全部测试过的函数作为Few-shot；\
    而如果输入的是一个包含了多个函数名称的list，则表示使用这些函数作为Few-shot。
    :param model: 可选参数，表示调用的Chat模型，默认选取glm-4；
    :param g: 可选参数，表示extract_function_code函数作用域，默认为globals()，即在当前操作空间全域内生效；
    :param detail: 可选参数，默认取值为0，还可以取值为1，表示extract_function_code函数打印新创建的外部函数细节；
    :return：新创建的函数名称。需要注意的是，在函数创建时，该函数也会在当前操作空间被定义，后续可以直接调用；
    """
    
    # 提取提示示例的函数名称
    if few_shot == 'all':
        few_shot_functions_name = show_functions(tested=True)
    elif type(few_shot) == list:
        few_shot_functions_name = few_shot
    # few_shot_functions = [globals()[name] for name in few_shot_functions_name]
    
    # 读取各阶段系统提示
    with open('./functions/tested functions/system_messages.json', 'r') as f:
        system_messages = json.load(f)
        
    # 各阶段提示message对象
    few_shot_messages_CM = []
    few_shot_messages_CD = []
    
    few_shot_messages = []
    
    # 先保存第一条消息，也就是system message
    few_shot_messages_CD += system_messages["system_message_CD"]
    few_shot_messages_CM += system_messages["system_message_CM"]
    few_shot_messages += system_messages["system_message"]

    # 创建不同阶段提示message
    for function_name in few_shot_functions_name:
        with open('./functions/tested functions/%s/%s_prompt.json' % (function_name, function_name), 'r') as f:
            msg = json.load(f)
        few_shot_messages_CD += msg["stage1_CD"]
        few_shot_messages_CM += msg["stage1_CM"]
        few_shot_messages += msg['stage2']
        
    # 读取用户需求，作为第一阶段CD环节User content
    new_req_CD_input = req
    few_shot_messages_CD.append({"role": "user", "content": new_req_CD_input})
    
    print('第一阶段CD环节提示创建完毕，正在进行CD提示...')
    
    # 第一阶段CD环节Chat模型调用过程
    response = client.chat.completions.create(
                        model=model,
                        messages=few_shot_messages_CD)
    new_req_pi = response.choices[0].message.content
    
    print('第一阶段CD环节提示完毕')
    
    # 第一阶段CM环节Messages创建
    new_req_CM_input = new_req_CD_input + new_req_pi
    few_shot_messages_CM.append({"role": "user", "content": new_req_CM_input})
    
    print('第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...')
    # 第一阶段CM环节Chat模型调用过程
    response = client.chat.completions.create(
                        model=model,
                        messages=few_shot_messages_CM)
    new_req_description = response.choices[0].message.content

    
    print('第一阶段CM环节提示完毕')
    
    # 第二阶段Messages创建过程
    few_shot_messages.append({"role": "user", "content": new_req_description})
    
    print('第二阶段提示创建完毕，正在进行第二阶段提示...')
    
    # 第二阶段Chat模型调用过程
    response = client.chat.completions.create(
                        model=model,
                        messages=few_shot_messages)
    new_req_function = response.choices[0].message.content
    
    
    print('第二阶段提示完毕，准备运行函数并编写提示示例')
    
    # 提取函数并运行，创建函数名称对象，统一都写入untested文件夹内
    function_name = extract_function_code(s=new_req_function, detail=detail, g=g)
    
    print('新函数保存在./functions/untested functions/%s/%s_module.py文件中' % (function_name, function_name))
    
    # 创建该函数提示示例
    new_req_messages_CD = [
                          {"role": "user", "content": new_req_CD_input},
                          {"role": "assistant", "content": new_req_pi}
                         ]
    new_req_messages_CM = [
                          {"role": "user", "content": new_req_CM_input},
                          {"role": "assistant", "content":new_req_description}
                         ]
    
    with open('./functions/untested functions/%s/%s_module.py' % (function_name, function_name), encoding='utf-8') as f:
        new_req_function = f.read()
    
    new_req_messages = [
                       {"role": "user", "content": new_req_description},
                       {"role": "assistant", "content":new_req_function}
                      ] 
    
    new_req_prompt = {
                     "stage1_CD": new_req_messages_CD,
                     "stage1_CM": new_req_messages_CM,
                     "stage2": new_req_messages
                    }   
    
    with open('./functions/untested functions/%s/%s_prompt.json' % (function_name, function_name), 'w') as f:
        json.dump(new_req_prompt, f)
        
    print('新函数提示示例保存在./functions/untested functions/%s/%s_prompt.json文件中' % (function_name, function_name))
    print('done')
    return function_name

In [442]:
function_name = 'get_latest_email'

In [457]:
with open('./functions/tested functions/%s/%s_prompt.json' % (function_name, function_name), 'r') as f:
    msg = json.load(f)

In [458]:
msg['stage2']

[{'role': 'user',
  'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
 {'role': 'assistant',
  'content': 'def get_latest_email(userId):\n    """\n    获取最新的一封邮件\n    参数:\n    user_id: 要检索邮件的用户的ID。默认值是\'me\'，数据格式为字符串类型。\n\n    返回:\n    一个列表，其中每个元素都是一个字典，表示一封邮件。每个字典包含以下键：\n    \'from_userid\': 发件人ID。\n    \'date\': 邮件的发送日期。\n    \'subject\': 邮件的主题。\n    \'snippet\': 邮件的摘要（前100个字符）。\n    """\n    mysql_pw="gpt"\n    \n    connection = pymysql.connect(\n    host=\'localhost\',  # 数据库地址\n    user=\'gpt\',  # 数据库用户名\n    passwd=mysql_pw,  # 数据库密码\n    db=\'my_mail\',  # 数据库名\n    charset=\'utf8\'  # 字符集选择utf8   \n    )\n    sql_query=f"SELECT * FROM mailbox WHERE userid = \'{user_id}\' ORDER BY date DESC LIMIT 1";\n\n    try:\n        with conne

In [533]:
few_shot_functions_name = show_functions(tested=True)

In [534]:
# 读取各阶段系统提示
with open('./functions/tested functions/system_messages.json', 'r') as f:
    system_messages = json.load(f)
        
# 各阶段提示message对象
few_shot_messages_CM = []
few_shot_messages_CD = []
few_shot_messages = []
    
# 先保存第一条消息，也就是system message
few_shot_messages_CD += system_messages["system_message_CD"]
few_shot_messages_CM += system_messages["system_message_CM"]
few_shot_messages += system_messages["system_message"]

In [535]:
for function_name in few_shot_functions_name:
    print(function_name)
    with open('./functions/tested functions/%s/%s_prompt.json' % (function_name, function_name), 'r') as f:
        msg = json.load(f)
    few_shot_messages_CD += msg["stage1_CD"]
    few_shot_messages_CM += msg["stage1_CM"]
    few_shot_messages += msg['stage2']

get_latest_email
get_email_count
retrieve_emails


In [536]:
few_shot_messages_CD

[{'role': 'system',
  'content': '# my_mail数据字典：my_mail数据库数据字典\n\n本数据字典记录了my_mail数据库中各张数据表的基本情况。\n我的my_mail中\nhost=\'localhost\',\nuser=\'gpt\',\npasswd="gpt",\ndb=\'my_mail\'\n\n## 1.mailbox数据表\n\n- 基本解释\n\n  \u200b\t\tmailbox数据表记录了邮箱里的信息，包含收件人，发件人，发件时间，邮件主题，邮件内容，邮件状态，邮件标签等。\n\n- 数据来源\n\n  \u200b\tmailbox数据集由系统记录，并且校验过相关信息，数据集的准确性和可信度都非常高。\n\n- 各字段说明\n\n| Column Name | Description | Value Range | Value Explanation | Type |\n|-------------|-------------|-------------|-------------------|------|\n| userid | 收件人 |              | 由字符串组成的 | VARCHAR(255) |\n| from_userid | 发件人 |  | 由字符串组成 | VARCHAR(255) |\n| date | 发邮件的时间 |  | 日期格式  | VARCHAR(255) |\n| subject | 邮件主题 |  | 由字符串组成 | VARCHAR(255) |\n| snippet | 邮件内容 | | 由字符串组成 | VARCHAR(255) |\n| status | 邮件状态 | 已读，未读 | 已读，未读 | VARCHAR(255) |\n| label | 邮件标签 |  | 由字符串组成| VARCHAR(255) |\n\n- 举个例子：\n| userid | from_userid | date | subject | snippet |status|label|\n|-------------|-------------|-------------|-------------------|------|--|--|\n| me

In [537]:
few_shot_messages_CM

[{'role': 'system',
  'content': '# my_mail数据字典：my_mail数据库数据字典\n\n本数据字典记录了my_mail数据库中各张数据表的基本情况。\n我的my_mail中\nhost=\'localhost\',\nuser=\'gpt\',\npasswd="gpt",\ndb=\'my_mail\'\n\n## 1.mailbox数据表\n\n- 基本解释\n\n  \u200b\t\tmailbox数据表记录了邮箱里的信息，包含收件人，发件人，发件时间，邮件主题，邮件内容，邮件状态，邮件标签等。\n\n- 数据来源\n\n  \u200b\tmailbox数据集由系统记录，并且校验过相关信息，数据集的准确性和可信度都非常高。\n\n- 各字段说明\n\n| Column Name | Description | Value Range | Value Explanation | Type |\n|-------------|-------------|-------------|-------------------|------|\n| userid | 收件人 |              | 由字符串组成的 | VARCHAR(255) |\n| from_userid | 发件人 |  | 由字符串组成 | VARCHAR(255) |\n| date | 发邮件的时间 |  | 日期格式  | VARCHAR(255) |\n| subject | 邮件主题 |  | 由字符串组成 | VARCHAR(255) |\n| snippet | 邮件内容 | | 由字符串组成 | VARCHAR(255) |\n| status | 邮件状态 | 已读，未读 | 已读，未读 | VARCHAR(255) |\n| label | 邮件标签 |  | 由字符串组成| VARCHAR(255) |\n\n- 举个例子：\n| userid | from_userid | date | subject | snippet |status|label|\n|-------------|-------------|-------------|-------------------|------|--|--|\n| me

In [538]:
few_shot_messages

[{'role': 'system',
  'content': '# my_mail数据字典：my_mail数据库数据字典\n\n本数据字典记录了my_mail数据库中各张数据表的基本情况。\n我的my_mail中\nhost=\'localhost\',\nuser=\'gpt\',\npasswd="gpt",\ndb=\'my_mail\'\n\n## 1.mailbox数据表\n\n- 基本解释\n\n  \u200b\t\tmailbox数据表记录了邮箱里的信息，包含收件人，发件人，发件时间，邮件主题，邮件内容，邮件状态，邮件标签等。\n\n- 数据来源\n\n  \u200b\tmailbox数据集由系统记录，并且校验过相关信息，数据集的准确性和可信度都非常高。\n\n- 各字段说明\n\n| Column Name | Description | Value Range | Value Explanation | Type |\n|-------------|-------------|-------------|-------------------|------|\n| userid | 收件人 |              | 由字符串组成的 | VARCHAR(255) |\n| from_userid | 发件人 |  | 由字符串组成 | VARCHAR(255) |\n| date | 发邮件的时间 |  | 日期格式  | VARCHAR(255) |\n| subject | 邮件主题 |  | 由字符串组成 | VARCHAR(255) |\n| snippet | 邮件内容 | | 由字符串组成 | VARCHAR(255) |\n| status | 邮件状态 | 已读，未读 | 已读，未读 | VARCHAR(255) |\n| label | 邮件标签 |  | 由字符串组成| VARCHAR(255) |\n\n- 举个例子：\n| userid | from_userid | date | subject | snippet |status|label|\n|-------------|-------------|-------------|-------------------|------|--|--|\n| me

In [539]:
new_req_CD_input = "请查下我的邮箱里是否有来自陆小凤的未读邮件，有的话请解读下这封未读邮件的内容。"

In [540]:
few_shot_messages_CD.append({"role": "user", "content": new_req_CD_input})
few_shot_messages_CD

[{'role': 'system',
  'content': '# my_mail数据字典：my_mail数据库数据字典\n\n本数据字典记录了my_mail数据库中各张数据表的基本情况。\n我的my_mail中\nhost=\'localhost\',\nuser=\'gpt\',\npasswd="gpt",\ndb=\'my_mail\'\n\n## 1.mailbox数据表\n\n- 基本解释\n\n  \u200b\t\tmailbox数据表记录了邮箱里的信息，包含收件人，发件人，发件时间，邮件主题，邮件内容，邮件状态，邮件标签等。\n\n- 数据来源\n\n  \u200b\tmailbox数据集由系统记录，并且校验过相关信息，数据集的准确性和可信度都非常高。\n\n- 各字段说明\n\n| Column Name | Description | Value Range | Value Explanation | Type |\n|-------------|-------------|-------------|-------------------|------|\n| userid | 收件人 |              | 由字符串组成的 | VARCHAR(255) |\n| from_userid | 发件人 |  | 由字符串组成 | VARCHAR(255) |\n| date | 发邮件的时间 |  | 日期格式  | VARCHAR(255) |\n| subject | 邮件主题 |  | 由字符串组成 | VARCHAR(255) |\n| snippet | 邮件内容 | | 由字符串组成 | VARCHAR(255) |\n| status | 邮件状态 | 已读，未读 | 已读，未读 | VARCHAR(255) |\n| label | 邮件标签 |  | 由字符串组成| VARCHAR(255) |\n\n- 举个例子：\n| userid | from_userid | date | subject | snippet |status|label|\n|-------------|-------------|-------------|-------------------|------|--|--|\n| me

第一阶段CD环节提示：

In [543]:
response = client.chat.completions.create(
                        model="glm-4",
                        messages=few_shot_messages_CD)

new_req_pi = response.choices[0].message.content

In [544]:
new_req_pi

'为了执行这个查询，我们需要编写一个SQL查询来检查邮箱中来自特定用户（在这个案例中是“陆小凤”）且状态为“未读”的邮件。以下是一个可能的SQL查询示例：\n\n```sql\nSELECT snippet\nFROM mailbox\nWHERE userid = \'me\' AND from_userid = \'陆小凤\' AND status = \'未读\'\nLIMIT 1;\n```\n\n这个查询假设`userid`字段代表收件人，`from_userid`代表发件人，并且`status`字段用来标识邮件是否已读。\n\n以下是按照您的请求，用伪代码表示的函数调用：\n\n```python\ndef check_and_read_unread_mail(email_account, sender, status):\n    # 假设已经建立了数据库连接，并且有一个cursor对象可以执行SQL查询。\n    query = """\n    SELECT snippet\n    FROM mailbox\n    WHERE userid = %s AND from_userid = %s AND status = %s\n    LIMIT 1;\n    """\n    cursor.execute(query, (email_account, sender, status))\n    result = cursor.fetchone()\n    if result:\n        # 如果有未读邮件，解读邮件内容\n        mail_content = result[0]\n        print("来自陆小凤的未读邮件内容如下：")\n        print(mail_content)\n        return True\n    else:\n        print("没有来自陆小凤的未读邮件。")\n        return False\n\n# 调用函数\nhas_unread_mail = check_and_read_unread_mail(\'me\', \'陆小凤\', \'未读\')\n```\n\n请注意，这个伪代码假设您已经有了一个数据库连接和游标（cursor）对象。在实

第一阶段CM环节提示：

In [545]:
new_req_CM_input = new_req_CD_input + new_req_pi
new_req_CM_input

'请查下我的邮箱里是否有来自陆小凤的未读邮件，有的话请解读下这封未读邮件的内容。为了执行这个查询，我们需要编写一个SQL查询来检查邮箱中来自特定用户（在这个案例中是“陆小凤”）且状态为“未读”的邮件。以下是一个可能的SQL查询示例：\n\n```sql\nSELECT snippet\nFROM mailbox\nWHERE userid = \'me\' AND from_userid = \'陆小凤\' AND status = \'未读\'\nLIMIT 1;\n```\n\n这个查询假设`userid`字段代表收件人，`from_userid`代表发件人，并且`status`字段用来标识邮件是否已读。\n\n以下是按照您的请求，用伪代码表示的函数调用：\n\n```python\ndef check_and_read_unread_mail(email_account, sender, status):\n    # 假设已经建立了数据库连接，并且有一个cursor对象可以执行SQL查询。\n    query = """\n    SELECT snippet\n    FROM mailbox\n    WHERE userid = %s AND from_userid = %s AND status = %s\n    LIMIT 1;\n    """\n    cursor.execute(query, (email_account, sender, status))\n    result = cursor.fetchone()\n    if result:\n        # 如果有未读邮件，解读邮件内容\n        mail_content = result[0]\n        print("来自陆小凤的未读邮件内容如下：")\n        print(mail_content)\n        return True\n    else:\n        print("没有来自陆小凤的未读邮件。")\n        return False\n\n# 调用函数\nhas_unread_mail = check_and_read_unread_mail(\'me\', \'陆小凤\', \'未读\')\n```\n\n

In [546]:
few_shot_messages_CM.append({"role": "user", "content": new_req_CM_input})
few_shot_messages_CM

[{'role': 'system',
  'content': '# my_mail数据字典：my_mail数据库数据字典\n\n本数据字典记录了my_mail数据库中各张数据表的基本情况。\n我的my_mail中\nhost=\'localhost\',\nuser=\'gpt\',\npasswd="gpt",\ndb=\'my_mail\'\n\n## 1.mailbox数据表\n\n- 基本解释\n\n  \u200b\t\tmailbox数据表记录了邮箱里的信息，包含收件人，发件人，发件时间，邮件主题，邮件内容，邮件状态，邮件标签等。\n\n- 数据来源\n\n  \u200b\tmailbox数据集由系统记录，并且校验过相关信息，数据集的准确性和可信度都非常高。\n\n- 各字段说明\n\n| Column Name | Description | Value Range | Value Explanation | Type |\n|-------------|-------------|-------------|-------------------|------|\n| userid | 收件人 |              | 由字符串组成的 | VARCHAR(255) |\n| from_userid | 发件人 |  | 由字符串组成 | VARCHAR(255) |\n| date | 发邮件的时间 |  | 日期格式  | VARCHAR(255) |\n| subject | 邮件主题 |  | 由字符串组成 | VARCHAR(255) |\n| snippet | 邮件内容 | | 由字符串组成 | VARCHAR(255) |\n| status | 邮件状态 | 已读，未读 | 已读，未读 | VARCHAR(255) |\n| label | 邮件标签 |  | 由字符串组成| VARCHAR(255) |\n\n- 举个例子：\n| userid | from_userid | date | subject | snippet |status|label|\n|-------------|-------------|-------------|-------------------|------|--|--|\n| me

In [547]:
response = client.chat.completions.create(
                        model="glm-4",
                        messages=few_shot_messages_CM)

new_req_description = response.choices[0].message.content

In [548]:
new_req_description

'以下是按照您的要求，编写的Python函数，该函数会查询数据库中来自特定发件人的未读邮件，并返回邮件内容。这个例子假设您已经有一个数据库连接对象（这里命名为`db_connection`），并且您的数据库环境已经设置好了。\n\n```python\nimport json\nimport pymysql  # 或者您使用的其他数据库连接库\n\n# 假设已经定义了数据库连接信息\nhost = \'localhost\'\nuser = \'gpt\'\npasswd = "gpt"\ndb = \'my_mail\'\n\n# 建立数据库连接\ndb_connection = pymysql.connect(host=host, user=user, passwd=passwd, db=db)\n\ndef check_and_read_unread_mail(user_id=\'me\', sender=\'陆小凤\'):\n    """\n    检查指定用户邮箱中来自特定发件人的未读邮件。\n\n    参数:\n    user_id (str): 收件人的用户ID，默认为\'me\'。\n    sender (str): 发件人的名称，默认为\'陆小凤\'。\n\n    返回:\n    json: 包含邮件内容的JSON对象。如果未找到邮件，则返回空JSON对象。\n\n    示例:\n    >>> check_and_read_unread_mail(\'me\', \'陆小凤\')\n    {\'snippet\': \'邮件内容\'}\n    """\n    try:\n        # 创建一个游标对象\n        with db_connection.cursor() as cursor:\n            # 执行SQL查询\n            query = """\n            SELECT snippet, subject, date\n            FROM mailbox\n            WHERE userid = %s AND from_userid = %s AND status = \'未读\'\n            ORDER BY date 

In [ ]:
第二阶段提示

In [549]:
few_shot_messages.append({"role": "user", "content": new_req_description})
few_shot_messages

[{'role': 'system',
  'content': '# my_mail数据字典：my_mail数据库数据字典\n\n本数据字典记录了my_mail数据库中各张数据表的基本情况。\n我的my_mail中\nhost=\'localhost\',\nuser=\'gpt\',\npasswd="gpt",\ndb=\'my_mail\'\n\n## 1.mailbox数据表\n\n- 基本解释\n\n  \u200b\t\tmailbox数据表记录了邮箱里的信息，包含收件人，发件人，发件时间，邮件主题，邮件内容，邮件状态，邮件标签等。\n\n- 数据来源\n\n  \u200b\tmailbox数据集由系统记录，并且校验过相关信息，数据集的准确性和可信度都非常高。\n\n- 各字段说明\n\n| Column Name | Description | Value Range | Value Explanation | Type |\n|-------------|-------------|-------------|-------------------|------|\n| userid | 收件人 |              | 由字符串组成的 | VARCHAR(255) |\n| from_userid | 发件人 |  | 由字符串组成 | VARCHAR(255) |\n| date | 发邮件的时间 |  | 日期格式  | VARCHAR(255) |\n| subject | 邮件主题 |  | 由字符串组成 | VARCHAR(255) |\n| snippet | 邮件内容 | | 由字符串组成 | VARCHAR(255) |\n| status | 邮件状态 | 已读，未读 | 已读，未读 | VARCHAR(255) |\n| label | 邮件标签 |  | 由字符串组成| VARCHAR(255) |\n\n- 举个例子：\n| userid | from_userid | date | subject | snippet |status|label|\n|-------------|-------------|-------------|-------------------|------|--|--|\n| me

In [550]:
response = client.chat.completions.create(
                model="glm-4",
                messages=few_shot_messages)

In [551]:
new_req_function = response.choices[0].message.content
new_req_function

'</div><div class="answer-title">Example Response</div><div class="answer-content">\n\n如果您需要返回最近的几封邮件的信息，以下是相应的Python函数实现：\n\n```python\nimport pymysql\nimport json\n\ndef get_recent_emails(n, user_id=\'me\'):\n    """\n    获取指定用户最近的几封邮件信息。\n\n    参数:\n    n (str): 要查询的邮件数量，必须为正整数。\n    user_id (str): 要检索邮件的用户的ID。默认值是\'me\'。\n\n    返回:\n    json: 返回一个JSON格式对象，包含查询到的邮件列表。\n          每封邮件包含以下信息：\'from_userid\', \'date\', \'subject\', \'snippet\'。\n\n    示例:\n    >>> get_recent_emails(\'5\', \'me\')\n    {\n        "emails": [\n            {"from_userid": "发件人ID", "date": "邮件日期", "subject": "邮件主题", "snippet": "邮件摘要"},\n            ...\n        ]\n    }\n    """\n    # 数据库连接配置\n    config = {\n        \'host\': \'localhost\',\n        \'user\': \'gpt\',\n        \'passwd\': \'gpt\',\n        \'db\': \'my_mail\',\n        \'charset\': \'utf8\'\n    }\n\n    try:\n        # 建立数据库连接\n        connection = pymysql.connect(**config)\n        with connection.cursor() as cursor:\n            # 查询最

In [552]:
extract_function_code(new_req_function, detail=1, g=globals())

{"emails": [{"from_userid": "西门吹雪", "date": "2024-06-07", "subject": "开会", "snippet": "明天10点到海淀区叶青大厦5楼502开会"}, {"from_userid": "西门吹雪", "date": "2024-06-06", "subject": "速回公司", "snippet": "赶快回来公司"}, {"from_userid": "陆小凤", "date": "2024-06-03", "subject": "2024年Q2规划", "snippet": "李先生，您好，我是陆小凤，这是我给您发的邮件，里面是Q2规划：聚焦核心业务，优化流程，提升效率。加强团队协作，培养人才。探索新市场，拓宽业务领域。确保目标达成，实现增长。详细见邮箱附件"}]}
def get_recent_emails(n, user_id='me'):
    """
    获取指定用户最近的几封邮件信息。

    参数:
    n (str): 要查询的邮件数量，必须为正整数。
    user_id (str): 要检索邮件的用户的ID。默认值是'me'。

    返回:
    json: 返回一个JSON格式对象，包含查询到的邮件列表。
          每封邮件包含以下信息：'from_userid', 'date', 'subject', 'snippet'。

    示例:
    >>> get_recent_emails('5', 'me')
    {
        "emails": [
            {"from_userid": "发件人ID", "date": "邮件日期", "subject": "邮件主题", "snippet": "邮件摘要"},
            ...
        ]
    }
    """
    # 数据库连接配置
    config = {
        'host': 'localhost',
        'user': 'gpt',
        'passwd': 'gpt',
        'db': 'my_mail',
        'charset': 'utf8'
    

'get_recent_emails'

接下来测试一下code_generate函数

In [561]:
few_shot_functions = ['get_latest_email', 'get_email_count']

In [562]:
req = "请查下我的邮箱里是否有来自陆小凤的未读邮件，有的话请解读下这封未读邮件的内容。"

In [563]:
function_name = code_generate(req=req, few_shot=few_shot_functions)

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
未读邮件内容： {"userid": "me", "from_userid": "\u9646\u5c0f\u51e4", "date": "2024-06-03", "subject": "2024\u5e74Q2\u89c4\u5212", "snippet": "\u674e\u5148\u751f\uff0c\u60a8\u597d\uff0c\u6211\u662f\u9646\u5c0f\u51e4\uff0c\u8fd9\u662f\u6211\u7ed9\u60a8\u53d1\u7684\u90ae\u4ef6\uff0c\u91cc\u9762\u662fQ2\u89c4\u5212\uff1a\u805a\u7126\u6838\u5fc3\u4e1a\u52a1\uff0c\u4f18\u5316\u6d41\u7a0b\uff0c\u63d0\u5347\u6548\u7387\u3002\u52a0\u5f3a\u56e2\u961f\u534f\u4f5c\uff0c\u57f9\u517b\u4eba\u624d\u3002\u63a2\u7d22\u65b0\u5e02\u573a\uff0c\u62d3\u5bbd\u4e1a\u52a1\u9886\u57df\u3002\u786e\u4fdd\u76ee\u6807\u8fbe\u6210\uff0c\u5b9e\u73b0\u589e\u957f\u3002\u8be6\u7ec6\u89c1\u90ae\u7bb1\u9644\u4ef6", "status": "\u672a\u8bfb", "label": "\u89c4\u5212"}
The function name is:check_unread_mail_from_luxiaofeng
新函数保存在./functions/untested functions/check_unread_mail_from_luxiaofeng/check_unrea

In [564]:
functions_list = [check_unread_mail_from_luxiaofeng]

In [565]:
messages = [{"role": "system", "content": "我的邮箱号叫me,陆小凤的邮箱号就叫陆小凤"},
            {"role": "user", "content": req}]

In [567]:
final_response = run_conversation(messages=messages, functions_list=functions_list, model="glm-4")

发生错误： Expecting value: line 15 column 29 (char 370)
正在重新运行...
{'check_unread_mail_from_luxiaofeng': <function check_unread_mail_from_luxiaofeng at 0x7f4df826c550>}
CompletionMessage(content='您的邮箱中确实有一封来自陆小凤的未读邮件，邮件的主题是“2024年Q2规划”。邮件的内容简要如下：\n\n陆小凤在邮件中提到，这是他给您发送的邮件，内容是关于第二季度的规划：聚焦核心业务，优化流程，提升效率。加强团队协作，培养人才。探索新市场，拓展业务领域。确保目标达成，实现增长。详细内容请查看邮箱附件。\n\n这封邮件目前的状态是“未读”，并且被标记为“规划”类别。如果您需要查看详细内容或附件，请登录您的邮箱进行查看。', role='assistant', tool_calls=None)


In [568]:
final_response

'您的邮箱中确实有一封来自陆小凤的未读邮件，邮件的主题是“2024年Q2规划”。邮件的内容简要如下：\n\n陆小凤在邮件中提到，这是他给您发送的邮件，内容是关于第二季度的规划：聚焦核心业务，优化流程，提升效率。加强团队协作，培养人才。探索新市场，拓展业务领域。确保目标达成，实现增长。详细内容请查看邮箱附件。\n\n这封邮件目前的状态是“未读”，并且被标记为“规划”类别。如果您需要查看详细内容或附件，请登录您的邮箱进行查看。'

## 4 额外需求测试

In [596]:
few_shot_functions = ['get_latest_email', 'get_email_count']

In [601]:
req = "请查下邮箱里，me这个邮箱给陆小凤发过多少封邮件？"

In [605]:
function_name = code_generate(req=req, few_shot=few_shot_functions)

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
{"count": 0}
The function name is:count_emails_sent_to_user
新函数保存在./functions/untested functions/count_emails_sent_to_user/count_emails_sent_to_user_module.py文件中
新函数提示示例保存在./functions/untested functions/count_emails_sent_to_user/count_emails_sent_to_user_prompt.json文件中
done


In [607]:
functions_list = [count_emails_sent_to_user]

In [608]:
messages = [{"role": "system", "content": "我的邮箱号叫me,陆小凤的邮箱号就叫陆小凤"},
            {"role": "user", "content": req}]

In [609]:
final_response = run_conversation(messages=messages, functions_list=functions_list, model="glm-4")

{'count_emails_sent_to_user': <function count_emails_sent_to_user at 0x7f4df86c5430>}
CompletionMessage(content='me这个邮箱给陆小凤发过0封邮件。', role='assistant', tool_calls=None)


In [610]:
final_response

'me这个邮箱给陆小凤发过0封邮件。'

In [611]:
req = "请查下邮箱里，陆小凤这个邮箱给我发过多少封邮件？"

In [612]:
messages = [{"role": "system", "content": "我的邮箱号叫me,陆小凤的邮箱号就叫陆小凤"},
            {"role": "user", "content": req}]

In [613]:
functions_list = [count_emails_sent_to_user]

In [614]:
final_response = run_conversation(messages=messages, functions_list=functions_list, model="glm-4")

{'count_emails_sent_to_user': <function count_emails_sent_to_user at 0x7f4df86c5430>}
CompletionMessage(content='陆小凤这个邮箱给您发过1封邮件。', role='assistant', tool_calls=None)


In [615]:
final_response

'陆小凤这个邮箱给您发过1封邮件。'

## 5. 总结

现在看起来稳定性的确比之前好了很多，但是在实际执行的过程当中，在编写课件过程当中，个别的方法我是运行了多次才获取到正确结果的
这个时候也给我们一个启示，就是大模型解决问题的时候，不一定一次性就正确，但是只要我们多运行几次往往就能解决问题
所以接下来如果想要让我们的项目更稳定，还需要实现自动Debug的功能。